In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pymongo

In [2]:
#mongo connection,
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.Blurbs
collection = db.sci_fi

In [4]:
def blurb_scrape(url):
    i = 0
    executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
    browser = Browser('chrome', **executable_path, headless = True)
    for i in range(0, 201, 2):
        try:
            browser.visit(url)
            link = browser.find_link_by_partial_href('/book/show/')[i]
            link.click()
        
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            div = soup.find('div', class_ = 'readable stacked')
            title = soup.find('h1').text
            author = soup.find('div', class_ = 'authorName__container').text
            blurb = div.find_all('span')
        except (AttributeError, IndexError):
            browser.back()
            continue
        if len(blurb) == 2:
            post = {
                "title" : title,
                "author" : author,
                'blurb' : blurb[1].text
            }
            collection.insert_one(post)
            browser.back()
        elif len(blurb) == 1:
            post = {
                "title" : title,
                "author" : author,
                'blurb' : blurb[0].text
            }
            collection.insert_one(post)
            browser.back()
        else:
            browser.back()

In [5]:
def loop_scrape(url):
    print("Loop Start")
    blurb_scrape(url)
    executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
    browser = Browser('chrome', **executable_path, headless = True)
    browser.visit(url)
    browser.is_element_present_by_text('Next', wait_time = 3)
    next_elem = browser.find_link_by_partial_text('Next').first
    next_elem.click()
    new_url = browser.url
    browser.quit()
    count = 0
    while (count < 100):
        count += 1
        print(f"Page {count} scraped.")
        try:
            blurb_scrape(new_url)
            executable_path = {'executable_path': '/home/brad/Documents/Novel-blurb-NLP/chromedriver'}
            browser = Browser('chrome', **executable_path, headless = True)
            browser.visit(new_url)
        
            browser.is_element_present_by_text('Next', wait_time = 3)
            next_elem = browser.find_link_by_partial_text('Next').first
            next_elem.click()
            new_url = browser.url
            browser.quit()
        except:
            break
  
        

In [6]:
collection = db.thriller
loop_scrape('https://www.goodreads.com/list/show/348.Thrillers?page=6')

Loop Start


ElementClickInterceptedException: Message: element click intercepted: Element <a title="The Burning Wire" href="/book/show/7099389-the-burning-wire">...</a> is not clickable at point (82, 11). Other element would receive the click: <img alt="Goodreads Choice Awards" class="siteHeader__topFullImage" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/siteheaderbannerimages/1572474810i/2._UX750_CR0,0,750,40_QL100_.jpg" srcset="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/siteheaderbannerimages/1572474810i/2._UX1500_CR0,0,1500,80_QL100_.jpg 2x" data-reactid=".1p7y4jucxs0.0.0.0.1">
  (Session info: headless chrome=78.0.3904.108)
